In [65]:
import numpy as np
import math 

In [66]:
# set random seed for reproducibility 
np.random.seed(42)

In [67]:
class Neuron:
    """
    A simple artificial neuron that computes a weighted sum of its inputs, applies an activation function, and produces an output.
    """

    def __init__(self, num_inputs):
        """
        Initialize the neuron with random weights and bias.
        """

        # xavier initialization for weights
        limit = 1/math.sqrt(num_inputs)
        self.weights = [np.random.uniform(-limit,limit) for _ in range(num_inputs)]

        # Bias initialization
        self.bias = np.random.uniform(-limit, limit)

        self.inputs = None
        self.output = None


    def forward(self, inputs):
        """
        Compute output of the neurons in the layer given the inputs.
        """

        # preserve copy of original inputs 
        self.inputs = inputs[:]

        # compute weighted sum
        weighted_sum = sum(w*x  for w,x in zip(self.weights, inputs)) + self.bias

        # applying ReLU by default
        self.output = max(0, weighted_sum)

        return self.output


    def __str__(self):
        """String representation of the neuron"""
        return f"Neuron(weights={[round(w, 3) for w in self.weights]}, bias={round(self.bias, 3)})"



In [68]:
class Layer:
    """
    A layer of neurons in a neural network.
    """

    def __init__(self, num_neurons, num_inputs_per_neuron, is_output=False):
        """
        Initialize the layer with given number of neurons, each with specified number of inputs.

        Args: 
            num_neurons (int): Number of neurons in the layer.
            num_inputs_per_neuron (int): Number of inputs each neuron receives.
            is_output (bool): Flag indicating if this layer is the output layer.
        """

        self.num_neurons = num_neurons
        self.num_inputs_per_neuron = num_inputs_per_neuron
        self.is_output = is_output

        # creating neurons for the layer
        self.neurons = [Neuron(num_inputs_per_neuron) for _ in range(num_neurons)]
        self.inputs = None
        self.outputs = None

        
    def forward(self, inputs):
        """
        Forwards pass through layers sequentially by computing outputs of all neurons in the layer.
        """

        self.inputs = inputs[:]

        # get output from each neuron
        self.outputs = [neuron.forward(inputs) for neuron in self.neurons]

        return self.outputs



    def __str__(self):
        """String representation of the layer"""
        layer_type = "Output" if self.is_output else "Hidden"
        return f"{layer_type} Layer ({self.num_neurons} neurons, {self.num_inputs_per_neuron} inputs each)"
        

In [69]:
class Network:
    """
        Neural Network consisting of multiple layers.
        Basic idea of Multi-Layer perceptron
    """

    def __init__(self, layer_sizes):

        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes)
        self. layers = []

        for i in range(1, self.num_layers):
            num_neurons = layer_sizes[i]
            num_inputs_per_neuron = layer_sizes[i-1]
            is_output = (i == self.num_layers - 1)

            layer = Layer(num_neurons, num_inputs_per_neuron, is_output)
            self.layers.append(layer)

    def forward(self, inputs):
        """
        Forward pass through the entire network.
        """

        for layer in self.layers:
            inputs = layer.forward(inputs)

        return inputs


In [70]:
nw = Network([2,3,1])

print("Network Architecture:")
for i, layer in enumerate(nw.layers):
    print(f"Layer {i+1}: {layer}")
    for j, neuron in enumerate(layer.neurons):
        print(f"  Neuron {j+1}: {neuron}")


Network Architecture:
Layer 1: Hidden Layer (3 neurons, 2 inputs each)
  Neuron 1: Neuron(weights=[-0.177, 0.637], bias=0.328)
  Neuron 2: Neuron(weights=[0.14, -0.486], bias=-0.486)
  Neuron 3: Neuron(weights=[-0.625, 0.518], bias=0.143)
Layer 2: Output Layer (1 neurons, 3 inputs each)
  Neuron 1: Neuron(weights=[0.24, -0.554, 0.543], bias=0.384)


In [71]:
input_data = [0.5, -1.5]
output = nw.forward(input_data)
print(f"Network output: {output}")

# You can also test layer by layer:
print("\nLayer by layer:")
layer1_output = nw.layers[0].forward(input_data)
print(f"Layer 1 output: {layer1_output}")

layer2_output = nw.layers[1].forward(layer1_output)
print(f"Layer 2 (final) output: {layer2_output}")

Network output: [0.21062314276556207]

Layer by layer:
Layer 1 output: [0, 0.3129595241608698, 0]
Layer 2 (final) output: [0.21062314276556207]


## Forward Pass:

Consider an architeure with 3 inputs, hidden1 with 3 neurons , hidden 2 with 2 neurons and output layer with 1 neuron.

prediction = sigma(W@X + b)

Where W is weights matrix, X is input vector, b is bias vector and sigma is activation function (ReLU here).

matrix form:
layer 1:
| w11 w12 w13 |     | x1 |     | b11 |
| w21 w22 w23 |  @  | x2 |  +  | b12 |
| w31 w32 w33 |     | x3 |     | b13 |
=>
o11 = sigma(w11*x1 + w12*x2 + w13*x3 + b11)
o21 = sigma(w21*x1 + w22*x2 + w23*x3 + b12)
o31 = sigma(w31*x1 + w32*x2 + w33*x3 + b13)

layer 2:
| w11 w12 |     | o11 |     | b21 |
| w21 w22 |  @  | o21 |  +  | b22 |
| w31 w32 |     | o31 |     
=>
o12 = sigma(w11*o11 + w12*o21 + b21)
o22 = sigma(w21*o11 + w22*o21 + b22)


layer 3:
|w11|     | o12 |     | b31 |
|w21|  @  | o22 |  +  
=> 
o31 (yhat) = sigma(w11*o12 + w21*o22 + b31)


**What is happening in forward function?**

- network.forward(X) takes input X and passes it through each layer sequentially.
- For each layer, layer.forward(inputs) it takes the current inputs, obtains outputs from all the neurons in that layer, and uses these outputs as inputs for the next layer.
- neuron.forward(inputs) computes the weighted sum of inputs plus bias, applies the activation function, and returns the output for that neuron.
- Finally, it returns the output from the last layer as the network's prediction.
